<a href="https://colab.research.google.com/github/sravs-2211/Task2_machinelearning_internshi/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("/content/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# PART 2: Feature Engineering
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Drop columns not needed
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# Create new features
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# Define target and features
X = df.drop('Survived', axis=1)
y = df['Survived']

# Define numerical and categorical columns
num_cols = ['Age', 'Fare', 'FamilySize']
cat_cols = ['Sex', 'Embarked', 'Pclass']

# Pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

# Combine all preprocessing
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Fit-transform the data
X_processed = preprocessor.fit_transform(X)
X_processed.shape  # Should return a transformed array


(891, 8)

In [7]:
# PART 3: Model Training & Tuning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, classification_report

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Define models
rf = RandomForestClassifier(random_state=42)
svc = SVC(probability=True, random_state=42)

# Hyperparameter grids
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5, 10]
}

svc_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

# Grid search
rf_grid = GridSearchCV(rf, rf_params, cv=5, scoring='f1')
svc_grid = GridSearchCV(svc, svc_params, cv=5, scoring='f1')

# Fit models
rf_grid.fit(X_train, y_train)
svc_grid.fit(X_train, y_train)

# Predictions
rf_best = rf_grid.best_estimator_
svc_best = svc_grid.best_estimator_

for model, name in zip([rf_best, svc_best], ['Random Forest', 'SVC']):
    y_pred = model.predict(X_test)
    print(f"\n----- {name} -----")
    print("F1 Score:", f1_score(y_test, y_pred))
    print("ROC-AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))



----- Random Forest -----
F1 Score: 0.7669172932330827
ROC-AUC: 0.8894465894465894
Confusion Matrix:
 [[97  8]
 [23 51]]
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.92      0.86       105
           1       0.86      0.69      0.77        74

    accuracy                           0.83       179
   macro avg       0.84      0.81      0.81       179
weighted avg       0.83      0.83      0.82       179


----- SVC -----
F1 Score: 0.7571428571428571
ROC-AUC: 0.8352638352638353
Confusion Matrix:
 [[92 13]
 [21 53]]
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.88      0.84       105
           1       0.80      0.72      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179

